In [ ]:
# import os
# import json
# import glob
# import yaml
# # ==========================================
# # CONFIGURATION
# # ==========================================
# FOLDER_PATH = "view_dependency_jsons"
# OUTPUT_HTML = "lineage_visualization.html"


# with open("target_object.yaml") as f:
#     target_object_yaml = yaml.safe_load(f)

# TARGET_OBJECT = target_object_yaml['target_tables'][0]

# # 2. Choose your Flow Direction:
# # "UPSTREAM"   = Reverse Lineage (What is this made from?)
# # "DOWNSTREAM" = Impact Analysis (What uses this?)
# DIRECTION = "DOWNSTREAM" 

# # ==========================================
# # 1. BUILD THE GRAPH (Both Directions)
# # ==========================================
# # forward_graph: Key = Source, Value = [Targets] (For Downstream)
# forward_graph = {}
# # reverse_graph: Key = Target, Value = [Sources] (For Upstream)
# reverse_graph = {}

# json_files = glob.glob(os.path.join(FOLDER_PATH, "*.json"))
# print(f"Loading {len(json_files)} lineage files...")

# for file_path in json_files:
#     try:
#         with open(file_path, 'r') as f:
#             relationships = json.load(f)
#             for item in relationships:
#                 src = item['source']
#                 tgt = item['target']
                
#                 # Build Forward (Source -> Target)
#                 if src not in forward_graph: forward_graph[src] = []
#                 if tgt not in forward_graph[src]: forward_graph[src].append(tgt)
                
#                 # Build Reverse (Target -> Source)
#                 if tgt not in reverse_graph: reverse_graph[tgt] = []
#                 if src not in reverse_graph[tgt]: reverse_graph[tgt].append(src)
                
#     except Exception as e:
#         print(f"Skipping broken file {file_path}: {e}")

# # ==========================================
# # 2. RECURSIVE TRACER
# # ==========================================
# def get_lineage_edges(node, direction, visited=None, edges=None):
#     if visited is None: visited = set()
#     if edges is None: edges = []
    
#     if node in visited:
#         return edges
#     visited.add(node)
    
#     # Select graph based on direction
#     if direction == "DOWNSTREAM":
#         # Look for children (Forward Graph)
#         relatives = forward_graph.get(node, [])
#     else:
#         # Look for parents (Reverse Graph)
#         relatives = reverse_graph.get(node, [])
    
#     for relative in relatives:
#         # Define edge tuple always as (Source, Target) for consistent arrow rendering
#         if direction == "DOWNSTREAM":
#             # Flow: Node -> Relative
#             edges.append((node, relative))
#         else:
#             # Flow: Relative -> Node (Data flows into current node)
#             edges.append((relative, node))
            
#         # Recurse
#         get_lineage_edges(relative, direction, visited, edges)
        
#     return edges

# # ==========================================
# # 3. GENERATE VISUALIZATION
# # ==========================================
# # Check if root exists in the relevant graph
# graph_to_check = forward_graph if DIRECTION == "DOWNSTREAM" else reverse_graph

# if TARGET_OBJECT in graph_to_check or TARGET_OBJECT in reverse_graph or TARGET_OBJECT in forward_graph:
#     edges = get_lineage_edges(TARGET_OBJECT, DIRECTION)
    
#     if not edges:
#         print(f"Object found, but has no {DIRECTION.lower()} dependencies.")
#     else:
#         # Generate Mermaid Syntax
#         mermaid_lines = ["graph LR"]
        
#         # Style the Root Node
#         safe_root = TARGET_OBJECT.replace(".", "_").replace("-", "_")
#         # Distinct color for the Root
#         mermaid_lines.append(f"    classDef rootNode fill:#ff0000,color:#fff,stroke:#333,stroke-width:4px;")
#         mermaid_lines.append(f"    {safe_root}:::{'rootNode'}")

#         for src, tgt in edges:
#             s_safe = src.replace(".", "_").replace("-", "_")
#             t_safe = tgt.replace(".", "_").replace("-", "_")
            
#             # Draw Arrow: Source --> Target
#             line = f'    {s_safe}["{src}"] --> {t_safe}["{tgt}"]'
#             mermaid_lines.append(line)

#         mermaid_content = "\n".join(mermaid_lines)

#         # HTML Template
#         html_template = f"""
#         <!DOCTYPE html>
#         <html>
#         <head>
#             <title>{DIRECTION} Lineage: {TARGET_OBJECT}</title>
#             <script src="https://cdn.jsdelivr.net/npm/mermaid/dist/mermaid.min.js"></script>
#             <script>mermaid.initialize({{startOnLoad:true}});</script>
#             <style>
#                 body {{ font-family: sans-serif; padding: 20px; }}
#                 h2 {{ color: #333; }}
#                 .badge {{ 
#                     background-color: #007bff; color: white; 
#                     padding: 5px 10px; border-radius: 4px; font-size: 0.8em; 
#                 }}
#             </style>
#         </head>
#         <body>
#             <h2>{TARGET_OBJECT} <span class="badge">{DIRECTION} FLOW</span></h2>
#             <div class="mermaid">
#     {mermaid_content}
#             </div>
#         </body>
#         </html>
#         """

#         with open(OUTPUT_HTML, "w", encoding="utf-8") as f:
#             f.write(html_template)

#         print(f"\n--- [Success] {DIRECTION} Graph generated: {OUTPUT_HTML} ---")
# else:
#     print(f"Error: Object '{TARGET_OBJECT}' not found in the lineage data.")